Importation de biblioteques

# Contexte
Le retard des vols est un problème majeur dans l'industrie aérienne qui impacte non seulement la satisfaction des passagers, mais aussi les coûts opérationnels des compagnies aériennes et des aéroports.

![Description de l'image](../notebooks/images/avion.jfif)


# Objectif
La tâche consiste à prédire le retard des vols (en minutes ou en heures) en fonction de plusieurs caractéristiques disponibles dans un jeu de données contenant des informations sur les vols.

In [6]:
import pandas as pd
pd.options.display.max_columns = None
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc = {'figure.figsize':(25,10)})

In [7]:
df_airlines = pd.read_csv("../data/airlines.csv")
df_aiports = pd.read_csv("../data/airports.csv")
df_flight = pd.read_csv("../data/flights.csv")

C:\Users\debgn\AppData\Local\Temp\ipykernel_9428\3853904211.py:3: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_flight = pd.read_csv("../data/flights.csv")


# Description des caractéristiques
Le jeu de données des retards de vol provient de la Federal Aviation Administration (FAA) et contient des informations sur les vols opérés aux États-Unis. 
Voici une description des principales caractéristiques présentes dans ces jeux de données

## jeux de données flights

In [8]:
df_flight.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN



Voici une description des caractéristiques de la dataset flights 

### **Informations sur la date et l'heure**

- <span style="color:blue;">*YEAR*</span> : Année de l’enregistrement du vol (utile pour analyser les tendances annuelles).
- <span style="color:blue;">*MONTH*</span> : Mois de l’enregistrement du vol (utile pour détecter des tendances saisonnières).
- <span style="color:blue;">*DAY*</span> : Jour du mois de l’enregistrement (utile pour identifier des motifs quotidiens ou mensuels).
- <span style="color:blue;">*DAY_OF_WEEK*</span> : Jour de la semaine (de 1 pour lundi à 7 pour dimanche), permet d’analyser les différences selon les jours.

### **Informations sur la compagnie et le vol**

- <span style="color:blue;">*AIRLINE*</span> : Code de la compagnie aérienne, permet d’identifier les compagnies responsables de potentiels retards.
- <span style="color:blue;">*FLIGHT_NUMBER*</span> : Numéro du vol, identifiant spécifique du vol.
- <span style="color:blue;">*TAIL_NUMBER*</span> : Numéro de queue de l’avion (identifiant unique pour suivre les performances des avions individuels).

### **Informations sur les aéroports**

- <span style="color:blue;">*ORIGIN_AIRPORT*</span> : Code de l’aéroport d’origine (peut influencer les retards dus aux infrastructures ou au trafic local).
- <span style="color:blue;">*DESTINATION_AIRPORT*</span> : Code de l’aéroport de destination (utile pour analyser les retards à l’arrivée).

### **Informations sur le départ**

- <span style="color:blue;">*SCHEDULED_DEPARTURE*</span> :  Heure planifiée de départ en minutes (format HHMM).
- <span style="color:blue;">*DEPARTURE_TIME*</span> : Heure réelle de départ en minutes (format HHMM).
- <span style="color:blue;">*DEPARTURE_DELAY*</span> :  Retard au départ en minutes (négatif si le vol part en avance).

### **Informations pendant le vol**

- <span style="color:blue;">*TAXI_OUT*</span> :  Temps de roulage au sol avant le décollage (en minutes).
- <span style="color:blue;">*WHEELS_OFF*</span> : Heure où l’avion quitte le sol (en minutes, format HHMM).
- <span style="color:blue;">*SCHEDULED_TIME*</span> : Durée prévue du vol (en minutes).
- <span style="color:blue;">*ELAPSED_TIME*</span> : emps total écoulé réel (en minutes).
- <span style="color:blue;">*AIR_TIME*</span> : Temps passé dans les airs (en minutes).
- <span style="color:blue;">*DISTANCE*</span> :  Distance parcourue pendant le vol (en miles).

### **Informations sur l’arrivée**

- <span style="color:blue;">*WHEELS_ON*</span> :  Heure où l’avion touche le sol à destination (en minutes, format HHMM).
- <span style="color:blue;">*TAXI_IN*</span> : Temps de roulage au sol après l’atterrissage (en minutes).
- <span style="color:blue;">*SCHEDULED_ARRIVAL*</span> :Heure planifiée d’arrivée en minutes (format HHMM).
- <span style="color:blue;">*ARRIVAL_TIME*</span> :  Heure réelle d’arrivée en minutes (format HHMM).
- <span style="color:blue;">*ARRIVAL_DELAY*</span> : Temps passé dans les airs (en minutes).
- <span style="color:blue;">*DISTANCE*</span> :  Retard à l’arrivée en minutes (négatif si le vol arrive en avance).

### **Informations sur les événements inhabituels**

- <span style="color:blue;">*DIVERTED*</span> :  Indique si le vol a été dérouté (1 pour oui, 0 pour non).
- <span style="color:blue;">*CANCELLED*</span> : Indique si le vol a été annulé (1 pour oui, 0 pour non)
- <span style="color:blue;">*CANCELLATION_REASON*</span> :Motif de l’annulation 
- <span style="color:blue;">*ARRIVAL_TIME*</span> :  Heure réelle d’arrivée en minutes (format HHMM).
- <span style="color:blue;">*ARRIVAL_DELAY*</span> : Temps passé dans les airs (en minutes).
- <span style="color:blue;">*DISTANCE*</span> :  Retard à l’arrivée en minutes (négatif si le vol arrive en avance).

### **Informations sur les types de retard**

- <span style="color:blue;">*AIR_SYSTEM_DELAY*</span> : Retard dû aux systèmes de contrôle du trafic aérien (en minutes).
- <span style="color:blue;">*SECURITY_DELAY*</span> : Retard  dû aux contrôles de sécurité (en minutes).
- <span style="color:blue;">*AIRLINE_DELAY*</span> :Retard  causé par la compagnie aérienne (en minutes). 
- <span style="color:blue;">*LATE_AIRCRAFT_DELAY*</span> :  Retard causé par l’arrivée tardive d’un autre avion (en minutes).
- <span style="color:blue;">*WEATHER_DELAY*</span> : Délai causé par des conditions météorologiques (en minutes).

## jeux de données Airlines

In [15]:
df_airlines.head(100)

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


Ce jeux de données est un complement de la base flights, elle sert à associé le nom de la compagnie au Code de la compagnie aérienne

- <span style="color:blue;">*IATA_CODE*</span> : Code unique de deux lettres attribué par l'IATA (International Air Transport Association) pour identifier les compagnies aériennes.
- <span style="color:blue;">*AIRLINE*</span> : Nom complet de la compagnie aérienne correspondant au code IATA.


## jeux de données Airports

In [10]:
df_aiports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


Cette base complete la base flights en ajouter le nom des aeroport, la ville, le pays ainsi que le coordonnées géographiques

- <span style="color:blue;">*IATA_CODE*</span> : Code unique de trois lettres attribué par l'IATA pour identifier un aéroport spécifique.
- <span style="color:blue;">*AIRPORT*</span> : Nom complet de l'aéroport.
- <span style="color:blue;">*CITY*</span> : Ville où se situe l'aéroport.
- <span style="color:blue;">*STATE*</span> : État (pour les États-Unis) où se trouve l'aéroport.
- <span style="color:blue;">*COUNTRY*</span> : Pays où se situe l'aéroport (dans ce dataset, il s'agit principalement des États-Unis).
- <span style="color:blue;">*LATITUDE*</span> : Coordonnée géographique indiquant la latitude de l'aéroport (utile pour la géolocalisation et l'analyse des distances).
- <span style="color:blue;">*LONGITUDE*</span> : Coordonnée géographique indiquant la longitude de l'aéroport (utile pour la géolocalisation et l'analyse des distances).


In [11]:
import pandas as pd

# 1. Merge pour ajouter les informations de la compagnie aérienne
df_flight = df_flight.merge(df_airlines[['IATA_CODE', 'AIRLINE']], 
                             how='left', 
                             left_on='AIRLINE', 
                             right_on='IATA_CODE', 
                             suffixes=('', '_AIRLINE'))

# 2. Merge pour ajouter les informations de l'aéroport d'origine
df_flight = df_flight.merge(df_aiports[['IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE']], 
                             how='left', 
                             left_on='ORIGIN_AIRPORT', 
                             right_on='IATA_CODE', 
                             suffixes=('_ORIGIN', '_ORIGIN_AIRPORT'))

# 3. Merge pour ajouter les informations de l'aéroport de destination
df_flight = df_flight.merge(df_aiports[['IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE']], 
                             how='left', 
                             left_on='DESTINATION_AIRPORT', 
                             right_on='IATA_CODE', 
                             suffixes=('_DESTINATION', '_DESTINATION_AIRPORT'))

# Affichage d'un aperçu des données fusionnées
print(df_flight.head())

# Sauvegarde dans un nouveau fichier CSV si nécessaire
df_flight.to_csv("../data/flights_merged.csv", index=False)


   YEAR  MONTH  DAY  DAY_OF_WEEK AIRLINE  FLIGHT_NUMBER TAIL_NUMBER  \
0  2015      1    1            4      AS             98      N407AS   
1  2015      1    1            4      AA           2336      N3KUAA   
2  2015      1    1            4      US            840      N171US   
3  2015      1    1            4      AA            258      N3HYAA   
4  2015      1    1            4      AS            135      N527AS   

  ORIGIN_AIRPORT DESTINATION_AIRPORT  SCHEDULED_DEPARTURE  DEPARTURE_TIME  \
0            ANC                 SEA                    5          2354.0   
1            LAX                 PBI                   10             2.0   
2            SFO                 CLT                   20            18.0   
3            LAX                 MIA                   20            15.0   
4            SEA                 ANC                   25            24.0   

   DEPARTURE_DELAY  TAXI_OUT  WHEELS_OFF  SCHEDULED_TIME  ELAPSED_TIME  \
0            -11.0      21.0        

In [16]:
df_flight.head(100)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,IATA_CODE_ORIGIN,AIRLINE_AIRLINE,IATA_CODE_ORIGIN_AIRPORT,AIRPORT_DESTINATION,CITY_DESTINATION,STATE_DESTINATION,COUNTRY_DESTINATION,LATITUDE_DESTINATION,LONGITUDE_DESTINATION,IATA_CODE,AIRPORT_DESTINATION_AIRPORT,CITY_DESTINATION_AIRPORT,STATE_DESTINATION_AIRPORT,COUNTRY_DESTINATION_AIRPORT,LATITUDE_DESTINATION_AIRPORT,LONGITUDE_DESTINATION_AIRPORT
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,PBI,Palm Beach International Airport,West Palm Beach,FL,USA,26.68316,-80.09559
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,US,US Airways Inc.,SFO,San Francisco International Airport,San Francisco,CA,USA,37.61900,-122.37484,CLT,Charlotte Douglas International Airport,Charlotte,NC,USA,35.21401,-80.94313
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AA,American Airlines Inc.,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,AS,Alaska Airlines Inc.,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2015,1,1,4,B6,2001,N358JB,BUF,JFK,535,530.0,-5.0,17.0,547.0,88.0,78.0,56.0,301,643.0,5.0,703,648.0,-15.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,B6,JetBlue Airways,BUF,Buffalo Niagara International Airport,Buffalo,NY,USA,42.94052,-78.73217,JFK,John F. Kennedy International Airport (New Yor...,New York,NY,USA,40.63975,-73.77893
96,2015,1,1,4,B6,2807,N190JB,PWM,JFK,535,528.0,-7.0,13.0,541.0,85.0,67.0,50.0,273,631.0,4.0,700,635.0,-25.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,B6,JetBlue Airways,PWM,Portland International Jetport,Portland,ME,USA,43.64617,-70.30875,JFK,John F. Kennedy International Airport (New Yor...,New York,NY,USA,40.63975,-73.77893
97,2015,1,1,4,B6,2023,N324JB,JFK,SJU,535,618.0,43.0,13.0,631.0,225.0,201.0,184.0,1598,1035.0,4.0,1020,1039.0,19.0,0,0,NaN,0.0,0.0,19.0,0.0,0.0,B6,JetBlue Airways,JFK,John F. Kennedy International Airport (New Yor...,New York,NY,USA,40.63975,-73.77893,SJU,Luis Muñoz Marín International Airport,San Juan,PR,USA,18.43942,-66.00183
98,2015,1,1,4,EV,4557,N12967,CRP,IAH,535,523.0,-12.0,12.0,535.0,61.0,52.0,33.0,201,608.0,7.0,636,615.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,EV,Atlantic Southeast Airlines,CRP,Corpus Christi International Airport,Corpus Christi,TX,USA,27.77036,-97.50122,IAH,George Bush Intercontinental Airport,Houston,TX,USA,29.98047,-95.33972


In [ ]:
# Fusionner df_flight avec df_airlines en utilisant la clé "AIRLINE"
df_flight_airlines = pd.merge(df_flight, df_airlines, how='left', left_on='AIRLINE', right_on='IATA_CODE')

# Fusionner df_flight_airlines avec df_airports pour les colonnes ORIGIN_AIRPORT et DESTINATION_AIRPORT
df_flight_airports = pd.merge(df_flight_airlines, df_aiports, how='left', left_on='ORIGIN_AIRPORT', right_on='IATA_CODE', suffixes=('_origin', '_destination'))

# Si vous souhaitez ajouter les informations sur l'aéroport de destination, vous pouvez procéder de manière similaire.
df_flight_airports = pd.merge(df_flight_airports, df_aiports, how='left', left_on='DESTINATION_AIRPORT', right_on='IATA_CODE', suffixes=('_origin', '_destination_final'))

# Afficher le résultat 
print(df_flight_airports.head())

   YEAR  MONTH  DAY  DAY_OF_WEEK AIRLINE_x  FLIGHT_NUMBER TAIL_NUMBER  \
0  2015      1    1            4        AS             98      N407AS   
1  2015      1    1            4        AA           2336      N3KUAA   
2  2015      1    1            4        US            840      N171US   
3  2015      1    1            4        AA            258      N3HYAA   
4  2015      1    1            4        AS            135      N527AS   

  ORIGIN_AIRPORT DESTINATION_AIRPORT  SCHEDULED_DEPARTURE  DEPARTURE_TIME  \
0            ANC                 SEA                    5          2354.0   
1            LAX                 PBI                   10             2.0   
2            SFO                 CLT                   20            18.0   
3            LAX                 MIA                   20            15.0   
4            SEA                 ANC                   25            24.0   

   DEPARTURE_DELAY  TAXI_OUT  WHEELS_OFF  SCHEDULED_TIME  ELAPSED_TIME  \
0            -11.0      